# A Note on this Notebook
This analyses data produced by running the `solve.sh` script in this directory.
Graphs are produced corresponding to data that was output.

# Problem Description

We try to enforce posterior consistency in the non-point-cloud case by redefining our objective functional

$$J''[u, q] = 
\underbrace{ N \int_{\Omega}\left(u_{interpolated} - u\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\alpha^2\int_\Omega|\nabla q|^2dx}_{\text{regularization}}$$

which is the same as $J'$ but where $J''_{\text{misfit}} = N \times J'_{\text{misfit}}$ to try to allow the misfit term to grow with number of measurements.

$\alpha = 0.02$ is used throughout.

# Setup

In [ ]:
import matplotlib.pyplot as plt
import firedrake

import numpy as np
from numpy import pi as π
from numpy import random

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import os, sys

import h5py

currentdir = os.path.dirname(os.path.realpath('__file__'))

In [ ]:
mesh = firedrake.UnitSquareMesh(32, 32)

# Solution Space
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)

# q (Control) Space
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

## Load $u_{true}$ and $q_{true}$

In [ ]:
u_true = firedrake.Function(V)
q_true = firedrake.Function(Q)

filename = os.path.join(currentdir, 'true-fields')
with firedrake.DumbCheckpoint(filename, mode=firedrake.FILE_READ) as chk:
    chk.load(q_true, name='q_true')
    chk.load(u_true, name='u_true')

# Plot Results

In [ ]:
num_points_set = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768, 65536, 131072, 262144]
methods = ['point-cloud', 'nearest', 'linear', 'clough-tocher', 'gaussian']
l2_q_norms = {method: [] for method in methods}
l2_u_norms = {method: [] for method in methods}

## Plot Fields and Save L2 Error Norms

In [ ]:
for num_points in num_points_set:

    # Test Loading and adjust plots as necessary
    filename = os.path.join(currentdir, 'observed-data.h5')
    try:
        with h5py.File(filename, 'r') as file:
            xs = file[f"xs_{num_points}"][:]
            u_obs_vals = file[f"u_obs_vals_{num_points}"][:]
            σ = firedrake.Constant(file[f"sigma_{num_points}"])
    except:
        # Can't load so move on
        continue
    methods_available = []        
    for method in methods:
        try:
            filename = os.path.join(currentdir, 'q-estimations')
            with firedrake.DumbCheckpoint(filename, mode=firedrake.FILE_READ) as chk:
                if method != 'point-cloud':
                    u_interpolated = firedrake.Function(V)
                    chk.load(u_interpolated, name=f'u_interpolated_{method}_{num_points}')
                q_min = firedrake.Function(Q)
                chk.load(q_min, name=f'q_min_{method}_{num_points}')
                q_err = firedrake.Function(Q)
                chk.load(q_err, name=f'q_err_{method}_{num_points}')
                methods_available.append(method)            
        except:
            pass
    if len(methods_available) == 0:
        # Nothing to plot so move on
        continue

    # Setup Plot
    ukw = {'vmin': 0.0, 'vmax': +0.2}
    kw = {'vmin': -4, 'vmax': +4, 'shading': 'gouraud'}
    title_fontsize = 20
    text_fontsize = 20
    fig, axes = plt.subplots(ncols=3, nrows=1+len(methods_available), sharex=True, sharey=True, figsize=(20,30), dpi=200)
    plt.suptitle('Estimating Log-Conductivity $q$ \n    where $k = k_0e^q$ and $-\\nabla \\cdot k \\nabla u = f$ for known $f$', fontsize=title_fontsize)
    for ax in axes.ravel():
        ax.set_aspect('equal')

    # Column 0
    axes[0, 0].set_title('$u_{true}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(u_true, axes=axes[0, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[0, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    # Column 1
    axes[0, 1].set_title('$q_{true}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_true, axes=axes[0, 1], **kw)
    cax = make_axes_locatable(axes[0, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    # Column 2
    axes[0, 2].set_title('$q_{true}-q_{true}$', fontsize=title_fontsize)
    zero_func = firedrake.Function(Q).assign(q_true-q_true)
    axes[0, 2].text(0.5, 0.5, f'$L^2$ Norm {firedrake.norm(zero_func, "L2"):.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(zero_func, axes=axes[0, 2], **kw);
    cax = make_axes_locatable(axes[0, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    for method_i, method in enumerate(methods_available):

        # Load fields
        filename = os.path.join(currentdir, 'q-estimations')
        with firedrake.DumbCheckpoint(filename, mode=firedrake.FILE_READ) as chk:
            if method != 'point-cloud':
                u_interpolated = firedrake.Function(V)
                chk.load(u_interpolated, name=f'u_interpolated_{method}_{num_points}')
            q_min = firedrake.Function(Q)
            chk.load(q_min, name=f'q_min_{method}_{num_points}')
            q_err = firedrake.Function(Q)
            chk.load(q_err, name=f'q_err_{method}_{num_points}')

        # Recalculate l2 norm error and save in l2_q_norms and l2_u_norms
        l2_q_norm = firedrake.norm(q_err, "L2")
        l2_q_norms[method].append((num_points, l2_q_norm))
        if method != 'point-cloud':
            u_err = firedrake.Function(Q).assign(u_interpolated-u_true)
            l2_u_norm = firedrake.norm(u_err, "L2")
            l2_u_norms[method].append((num_points, l2_u_norm))

        row = method_i+1

        # column 0
        if method == 'point-cloud':
            axes[row, 0].set_title('Sampled Noisy $u_{obs}$', fontsize=title_fontsize)
            colors = axes[row, 0].scatter(xs[:, 0], xs[:, 1], c=u_obs_vals, vmin=0.0, vmax=0.2)
        else:
            axes[row, 0].set_title(f'$u_{{interpolated}}^{{{method}}}$', fontsize=title_fontsize)
            colors = firedrake.tripcolor(u_interpolated, axes=axes[row, 0], shading='gouraud', **ukw)
        cax = make_axes_locatable(axes[row, 0]).append_axes("right", size="5%", pad=0.05)
        fig.colorbar(colors, cax=cax)

        # column 1
        if method == 'point-cloud':
            axes[row, 1].set_title('$q_{est}$ from $u_{obs}$', fontsize=title_fontsize)
        else:
            axes[row, 1].set_title(f'$q_{{est}}^{{{method}}}$ from $u_{{interpolated}}^{{{method}}}$', fontsize=title_fontsize)
        colors = firedrake.tripcolor(q_min, axes=axes[row, 1], **kw)
        cax = make_axes_locatable(axes[row, 1]).append_axes("right", size="5%", pad=0.05)
        fig.colorbar(colors, cax=cax)

        # column 2
        axes[row, 2].set_title('$q_{est}-q_{true}$', fontsize=title_fontsize)
        axes[row, 2].text(0.5, 0.5, f'$L^2$ Norm {l2_q_norm:.2f}', ha='center', fontsize=text_fontsize)
        colors = firedrake.tripcolor(q_err, axes=axes[row, 2], **kw);
        cax = make_axes_locatable(axes[row, 2]).append_axes("right", size="5%", pad=0.05)
        fig.colorbar(colors, cax=cax)

    # Save figure
    plt.savefig(f'posterior-consistency-{num_points}-pts.png')
        
    # Close to save memory
    plt.close(fig)


## Plot L2 Errors

In [ ]:
cmap = plt.get_cmap("tab10")
fig, ax = plt.subplots(dpi=200)
ax.set_xscale('log', basex=2)
for i, method in enumerate(methods):
    arr = np.asarray(l2_q_norms[method])
    method_num_points = arr[:,0]
    method_l2_q_norms = arr[:,1]
    ax.plot(method_num_points, method_l2_q_norms, marker='o', color=cmap(i))
ax.legend(methods, title='Method')
ax.set_xlabel('Number of Points N')
ax.set_ylabel('$||q_{est}-q_{true}||_{L^2}$')
ax.set_title('Estimating Log-Conductivity $q$ \n    where $k = k_0e^q$ and $-\\nabla \\cdot k \\nabla u = f$ for known $f$')
plt.savefig(f'l2_q_norms.png')

In [ ]:
cmap = plt.get_cmap("tab10")
fig, ax = plt.subplots(dpi=200)
ax.set_xscale('log', basex=2)
valid_methods = []
for i, method in enumerate(methods):
    arr = np.asarray(l2_u_norms[method])
    if len(arr.shape) != 2:
        continue
    valid_methods.append(method)
    method_num_points = arr[:,0]
    method_l2_u_norms = arr[:,1]
    ax.plot(method_num_points, method_l2_u_norms, marker='o', color=cmap(i))
ax.legend(valid_methods, title='Method')
ax.set_xlabel('Number of Points N')
ax.set_ylabel('$||u_{interpolated}-u_{true}||_{L^2}$')
ax.set_title('Estimating Log-Conductivity $q$ \n    where $k = k_0e^q$ and $-\\nabla \\cdot k \\nabla u = f$ for known $f$')
plt.savefig(f'l2_u_norms.png')